In [58]:
from IPython.display import clear_output

In [59]:
import pandas as pd
import numpy as np

import json
import ast
import boto3
from sagemaker import get_execution_role

from sklearn.metrics.pairwise import cosine_similarity, euclidean_distances
from sklearn.decomposition import TruncatedSVD

READ IN INPUT DATA FROM s3

In [60]:
role = get_execution_role()
bucket='sagemaker-us-east-1-770526846351'

In [61]:
train_location = 's3://{}/{}'.format(bucket, 'train.csv')
test_location = 's3://{}/{}'.format(bucket, 'test.csv')

df_train = pd.read_csv(train_location)
df_test = pd.read_csv(test_location)

In [62]:
df = pd.concat([df_train, df_test])

In [63]:
investor_location = 's3://{}/{}'.format(bucket, 'investor_sim_input.csv')
transaction_location = 's3://{}/{}'.format(bucket, 'transaction_sim_input.csv')

df_investor = pd.read_csv(investor_location)
df_txn = pd.read_csv(transaction_location)

In [64]:
df_investor = df_investor.set_index('buyerId')
df_txn = df_txn.set_index('id')

In [65]:
df_txn = pd.merge(df_txn, df[['id', 'test']], on=['id'], how='inner')

In [66]:
df_txn.head()

,id,0,1,2,3,4,5,6,7,test
0,39113-20210226-3644-0,-0.301298,-0.546314,-0.104104,0.066825,-0.007681,0.039699,0.020044,-0.011568,0.0
1,39113-20210327-725-0,-0.303966,-0.560204,-0.100781,0.074098,-0.017417,0.046923,0.024052,-0.010863,0.0
2,39113-20220504-71-0,-0.212017,-0.154348,-0.122105,-0.015650,0.127444,-0.033579,-0.033286,-0.016304,0.0
3,39113-20210626-514-0,-0.306268,-0.567259,-0.106608,0.069344,-0.012742,0.041548,0.021907,-0.011514,0.0
4,39113-20220725-450-0,-0.199218,-0.102462,-0.115829,-0.018570,0.136025,-0.036113,-0.036573,-0.016369,0.0


SAMPLE (FOR TXNS)

In [10]:
# df_txn_sample = df_txn.groupby('test', group_keys=False).apply(lambda x: x.sample(frac=0.03))

In [11]:
# df_txn_sample = df_txn_sample.drop(['test'], axis=1)

In [12]:
# df_txn_sample = df_txn_sample.set_index('id')

In [13]:
# df_txn_sample.shape

In [14]:
# df_txn_sample.head()

TRANSACTION - COSINE SIMILARITY

In [15]:
# txn_sim_matrix = cosine_similarity(df_txn_sample)

In [16]:
# txn_sim = pd.DataFrame(txn_sim_matrix, columns=df_txn_sample.index, index=df_txn_sample.index)

In [17]:
# txn_sim.head()

In [18]:
# txn_sim.memory_usage(deep=True).sum()/1000000000.0

In [19]:
# txn_result = {'id':[], 'recs':[]}

# for idx, row in txn_sim.iterrows():
#     data = txn_sim[idx].sort_values(ascending=False).reset_index(name='cosine_score').head(6).tail(5).to_dict('records')
#     txn_result['id'].append(idx)
#     txn_result['recs'].append(data)

In [20]:
# df_txn_result = pd.DataFrame(txn_result)

In [21]:
# df_txn_result.head()

In [22]:
# df_txn_result.to_csv('s3://{}/{}'.format(bucket, 'transaction_cosine_sim_result.csv'))

TRANSACTION - EUCLIDEAN

In [23]:
# txn_eu_sim_matrix = euclidean_distances(df_txn_sample)

In [24]:
# txn_eu_sim = pd.DataFrame(txn_eu_sim_matrix, columns=df_txn_sample.index, index=df_txn_sample.index)

In [25]:
# txn_eu_sim.head()

In [26]:
# txn_eu_result = {'id':[], 'recs':[]}

# for idx, row in txn_eu_sim.iterrows():
#     data = txn_eu_sim[idx].sort_values(ascending=True).reset_index(name='euclidean_distance').head(6).tail(5).to_dict('records')
#     txn_eu_result['id'].append(idx)
#     txn_eu_result['recs'].append(data)

In [27]:
# df_txn_eu_result = pd.DataFrame(txn_eu_result)

In [28]:
# df_txn_eu_result.head()

In [29]:
# df_txn_eu_result.to_csv('s3://{}/{}'.format(bucket, 'transaction_euclidean_sim_result.csv'))

INVESTORS - COSINE

In [67]:
df_investor.shape

(17650, 20)

In [68]:
inv_sim_matrix = cosine_similarity(df_investor)

In [69]:
inv_sim = pd.DataFrame(inv_sim_matrix, columns=df_investor.index, index=df_investor.index)

In [70]:
inv_sim.head()

buyerId,00074b1019380cc21471e2a46825c1fe,0007fba1809a022de16900e96add9515,0015cc309601fe8731febcc98fc7f6c2,001a5216b4a83da8dd43f76628258482,001bcb3fd35ccd4829fe2ebc7caeb142,002332ae8a612bdc3fbf78c98815afe4,0028484c48ed26f0b73caef882961cab,002e8cf8b2ca3d11c32412359e745c9c,0036f3877643d85c8194950565bc3517,003fb17e4d126dbec5a293facbed2210,...,ffd680795a380a4c35e200aef3357d80,ffd78b241cd95c113cf294211df954a2,ffe6d9c4a1a4f3fc65f15e011ad05d02,ffe6e23923627b67888e38941ac06aad,ffeb0fd7120ff6033c6115aa191b5835,ffebf39a17d11d62989632ca21db8727,fff12327cbe35326fdbc477345b6af79,fff540d19e154474992853d62709846e,fff918f6975aeebb75ca4b64727bb989,fffa6b9a171611c1c04039318f81cdf6
buyerId,,,,,,,,,,,,,,,,,,,,,
00074b1019380cc21471e2a46825c1fe,1.000000,0.044892,0.205812,-0.211854,0.257362,-0.014501,-0.372045,0.042619,0.089257,-0.000117,...,-0.449192,0.142505,-0.541869,0.208961,-0.036234,-0.176600,-0.019108,-0.188713,-0.213455,0.332207
0007fba1809a022de16900e96add9515,0.044892,1.000000,0.640293,-0.680210,0.073118,-0.420529,-0.323810,-0.165774,0.432671,0.673981,...,0.296743,0.538509,0.204224,0.593982,-0.298573,-0.728214,-0.371327,-0.603834,-0.124945,0.608118
0015cc309601fe8731febcc98fc7f6c2,0.205812,0.640293,1.000000,-0.901714,-0.014295,-0.647725,-0.260297,-0.421844,0.685231,0.877613,...,0.235883,0.799204,0.190930,0.965939,-0.583992,-0.893408,-0.631174,-0.795986,0.176729,0.900367
001a5216b4a83da8dd43f76628258482,-0.211854,-0.680210,-0.901714,1.000000,0.247036,0.795857,0.334627,0.538867,-0.554224,-0.933717,...,-0.310702,-0.843218,-0.194063,-0.937814,0.698302,0.983147,0.744815,0.958382,-0.202644,-0.904770
001bcb3fd35ccd4829fe2ebc7caeb142,0.257362,0.073118,-0.014295,0.247036,1.000000,0.738133,0.172194,0.845882,0.105211,-0.217263,...,-0.149891,0.092580,0.138939,-0.111288,0.764730,0.159331,0.731106,0.302053,-0.184464,0.047801


In [71]:
inv_sim['00074b1019380cc21471e2a46825c1fe'].sort_values(ascending=False).head(6)

buyerId
00074b1019380cc21471e2a46825c1fe    1.000000
d70f1f818b2ae4bb8e9d7ca0bf9482c1    0.999532
ee3bb2cd4939a7fee42456a4689655da    0.999506
81dd7903a915c913e12d7111d0d8aead    0.999110
f1fb53c2aed1c16786d7e7f10199748e    0.999093
137dbc2ce89537f1ed1a2354d7ee55b8    0.999086
Name: 00074b1019380cc21471e2a46825c1fe, dtype: float64

In [72]:
inv_result = {'buyerId':[], 'recs':[]}

for idx, row in inv_sim.iterrows():
    data = inv_sim[idx].sort_values(ascending=False).reset_index(name='cosine_score').head(11).tail(10).to_dict('records')
    inv_result['buyerId'].append(idx)
    inv_result['recs'].append(data)

In [73]:
df_inv_result = pd.DataFrame(inv_result)

In [75]:
df_inv_result.head(10)

,buyerId,recs
0,00074b1019380cc21471e2a46825c1fe,[{'buyerId': 'd70f1f818b2ae4bb8e9d7ca0bf9482c1...
1,0007fba1809a022de16900e96add9515,[{'buyerId': 'f0a7a165a818a7c7b0773bdd158b75c4...
2,0015cc309601fe8731febcc98fc7f6c2,[{'buyerId': 'c55c9fdfe4c8f1dda3dc64be67d23eb1...
3,001a5216b4a83da8dd43f76628258482,[{'buyerId': '54242d7fd6b86a032aeb68cfb3639026...
4,001bcb3fd35ccd4829fe2ebc7caeb142,[{'buyerId': 'd5fd764377cb4c0b448a117671832299...
5,002332ae8a612bdc3fbf78c98815afe4,[{'buyerId': '6dce7ed95b2ff0bc420aa66c86c8bb54...
6,0028484c48ed26f0b73caef882961cab,[{'buyerId': '0fbc1d223fbbb6459cdd74c5b131eb7b...
7,002e8cf8b2ca3d11c32412359e745c9c,[{'buyerId': 'ac2fdb74000ec4ffdd016220c536d4bf...
8,0036f3877643d85c8194950565bc3517,[{'buyerId': 'f809686977c06165789a37a68932f830...
9,003fb17e4d126dbec5a293facbed2210,[{'buyerId': 'c7c431fe57e65688fb28b7cb53ce1d83...


In [76]:
df_inv_result.to_csv('s3://{}/{}'.format(bucket, 'investor_cosine_sim_result_top10.csv'))

INVESTORS - EUCLIDEAN

In [77]:
inv_eu_sim_matrix = euclidean_distances(df_investor)

In [78]:
inv_eu_sim = pd.DataFrame(inv_eu_sim_matrix, columns=df_investor.index, index=df_investor.index)

In [80]:
inv_eu_sim.head(10)

buyerId,00074b1019380cc21471e2a46825c1fe,0007fba1809a022de16900e96add9515,0015cc309601fe8731febcc98fc7f6c2,001a5216b4a83da8dd43f76628258482,001bcb3fd35ccd4829fe2ebc7caeb142,002332ae8a612bdc3fbf78c98815afe4,0028484c48ed26f0b73caef882961cab,002e8cf8b2ca3d11c32412359e745c9c,0036f3877643d85c8194950565bc3517,003fb17e4d126dbec5a293facbed2210,...,ffd680795a380a4c35e200aef3357d80,ffd78b241cd95c113cf294211df954a2,ffe6d9c4a1a4f3fc65f15e011ad05d02,ffe6e23923627b67888e38941ac06aad,ffeb0fd7120ff6033c6115aa191b5835,ffebf39a17d11d62989632ca21db8727,fff12327cbe35326fdbc477345b6af79,fff540d19e154474992853d62709846e,fff918f6975aeebb75ca4b64727bb989,fffa6b9a171611c1c04039318f81cdf6
buyerId,,,,,,,,,,,,,,,,,,,,,
00074b1019380cc21471e2a46825c1fe,0.000000,2.355197,2.280446,3.822196,2.070148,2.198707,2.230008,2.169062,2.627589,2.394941,...,2.275416,2.177321,3.019449,2.227193,2.263294,4.421440,2.215482,2.637708,4.761532,2.018453
0007fba1809a022de16900e96add9515,2.355197,0.000000,1.072386,3.597977,1.129288,1.401213,1.203402,1.291014,1.607827,0.887903,...,1.055669,0.964374,1.508388,1.059532,1.467868,4.386818,1.418436,2.068564,4.114677,0.890959
0015cc309601fe8731febcc98fc7f6c2,2.280446,1.072386,0.000000,4.026604,1.406589,1.741038,1.443648,1.644190,1.278845,0.661297,...,1.324935,0.854402,1.680519,0.362476,1.853797,4.773657,1.778171,2.433734,3.829742,0.926993
001a5216b4a83da8dd43f76628258482,3.822196,3.597977,4.026604,0.000000,2.691937,2.365278,2.683774,2.516504,3.990778,3.796206,...,2.857729,3.497994,3.252963,3.933893,2.324679,0.952640,2.360841,1.636214,5.148956,3.227419
001bcb3fd35ccd4829fe2ebc7caeb142,2.070148,1.129288,1.406589,2.691937,0.000000,0.350942,0.371219,0.290471,1.736608,1.208905,...,0.469771,0.879469,1.273703,1.310850,0.488467,3.479293,0.398331,1.148942,3.901911,0.597246
002332ae8a612bdc3fbf78c98815afe4,2.198707,1.401213,1.741038,2.365278,0.350942,0.000000,0.479733,0.190247,1.988730,1.514250,...,0.643629,1.161651,1.333162,1.633773,0.215054,3.171533,0.110483,0.882328,3.950951,0.907872
0028484c48ed26f0b73caef882961cab,2.230008,1.203402,1.443648,2.683774,0.371219,0.479733,0.000000,0.495586,1.848274,1.141974,...,0.254562,0.894698,1.230873,1.290048,0.652824,3.450677,0.536683,1.157925,3.711733,0.576610
002e8cf8b2ca3d11c32412359e745c9c,2.169062,1.291014,1.644190,2.516504,0.290471,0.190247,0.495586,0.000000,1.932277,1.419252,...,0.611907,1.033862,1.234229,1.535044,0.219513,3.334013,0.165526,1.058339,3.943090,0.823850
0036f3877643d85c8194950565bc3517,2.627589,1.607827,1.278845,3.990778,1.736608,1.988730,1.848274,1.932277,0.000000,1.476512,...,1.775669,1.513507,1.927230,1.492228,2.088906,4.716993,2.046667,2.470584,4.394512,1.549443


In [81]:
# inv_eu_sim['00074b1019380cc21471e2a46825c1fe'].sort_values(ascending=True).head(6)

In [82]:
# test_sim = inv_eu_sim['00074b1019380cc21471e2a46825c1fe'].reset_index()

In [83]:
# test_sim_df = pd.DataFrame(test_sim)

In [84]:
# test_min = min(test_sim_df['00074b1019380cc21471e2a46825c1fe'])
# test_max = max(test_sim_df['00074b1019380cc21471e2a46825c1fe'])

In [85]:
# test_sim_df['norm'] = (test_sim_df['00074b1019380cc21471e2a46825c1fe'] - test_min) / (test_max - test_min)

In [86]:
# test_sim_df['similarity'] = 1 - test_sim_df['norm']

In [87]:
# test_sim_df.sort_values(by='similarity',ascending=False).head(6)

In [88]:
inv_eu_result = {'buyerId':[], 'recs':[]}

for idx, row in inv_eu_sim.iterrows():
    data = inv_eu_sim[idx].sort_values(ascending=True).reset_index(name='euclidean_distance').head(11).tail(10).to_dict('records')
    inv_eu_result['buyerId'].append(idx)
    inv_eu_result['recs'].append(data)

In [89]:
df_inv_eu_result = pd.DataFrame(inv_eu_result)

In [90]:
df_inv_eu_result.head()

,buyerId,recs
0,00074b1019380cc21471e2a46825c1fe,[{'buyerId': '137dbc2ce89537f1ed1a2354d7ee55b8...
1,0007fba1809a022de16900e96add9515,[{'buyerId': 'b271250149da9a9e28d8b9607b82cf04...
2,0015cc309601fe8731febcc98fc7f6c2,[{'buyerId': '8b875f3af020a56a637278489ba409bc...
3,001a5216b4a83da8dd43f76628258482,[{'buyerId': '54242d7fd6b86a032aeb68cfb3639026...
4,001bcb3fd35ccd4829fe2ebc7caeb142,[{'buyerId': '40bd82f2ce5181169530fad621e7a0aa...


In [91]:
df_inv_eu_result.to_csv('s3://{}/{}'.format(bucket, 'investor_euclidean_sim_result_top10.csv'))